빅데이터 수집시스템 구성하기

In [271]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
from glob import glob
import warnings
warnings.filterwarnings('ignore')

In [272]:
url = 'https://minwon.police.go.kr/#customerCenter/plcstSearch'
driver = webdriver.Chrome()
driver.get(url)


In [273]:
driver.find_element_by_css_selector('#tab3_1').click()



In [274]:
driver.find_element_by_css_selector('#indexedA').click()

In [275]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
trs=soup.select('#minwonTbody > tr ')
# 이거때문에 2시간 고민했다 까먹지말자..
len(trs)




5

In [276]:
tr=trs[0]
name= tr.find('td').get_text().strip()
name

'경찰청 서울특별시경찰청 서울중부경찰서'

In [277]:
addr=tr.select_one('.title').get_text().strip()
addr

'서울특별시 중구 수표로 27'

In [278]:
lines=[]
for tr in trs:
        name =tr.select_one('td').get_text().strip()
        addr=tr.select_one('.title').get_text().strip()
        lines.append([name,addr])

lines


[['경찰청 서울특별시경찰청 서울중부경찰서', '서울특별시 중구 수표로 27'],
 ['경찰청 서울특별시경찰청 서울종로경찰서', '서울특별시 종로구 율곡로 46'],
 ['경찰청 서울특별시경찰청 서울남대문경찰서', '서울특별시 중구 한강대로 410'],
 ['경찰청 서울특별시경찰청 서울서대문경찰서', '서울특별시 서대문구 통일로 113'],
 ['경찰청 서울특별시경찰청 서울혜화경찰서', '서울특별시 종로구 창경궁로 112-16']]

In [279]:
for _ in range(6):
    driver.find_element_by_css_selector('.p.next').click()
    time.sleep(1)
    soup = BeautifulSoup(driver.page_source,'html.parser')
    trs = soup.select('#minwonTbody > tr')
    for tr in trs:
        name = tr.find('td').get_text().strip().split()[-1]
        addr = tr.select_one('.title').get_text().strip()
        lines.append([name,addr])

In [243]:
driver.close()

In [280]:
df = pd.DataFrame(lines,columns=['경찰서명','주소'])
df.head(30)

,경찰서명,주소
0,경찰청 서울특별시경찰청 서울중부경찰서,서울특별시 중구 수표로 27
1,경찰청 서울특별시경찰청 서울종로경찰서,서울특별시 종로구 율곡로 46
2,경찰청 서울특별시경찰청 서울남대문경찰서,서울특별시 중구 한강대로 410
3,경찰청 서울특별시경찰청 서울서대문경찰서,서울특별시 서대문구 통일로 113
4,경찰청 서울특별시경찰청 서울혜화경찰서,서울특별시 종로구 창경궁로 112-16
5,서울용산경찰서,서울특별시 용산구 원효로89길 24
6,서울성북경찰서,서울특별시 성북구 보문로 170
7,서울동대문경찰서,서울특별시 동대문구 약령시로 21길 29
8,서울마포경찰서,서울특별시 마포구 마포대로 183
9,서울영등포경찰서,서울특별시 영등포구 국회대로 608


In [294]:
df[df.경찰서명.isin(["서울서부경찰서","서울강동경찰서"])].index

Int64Index([13, 20], dtype='int64')

In [301]:
df.주소[13]='서울특별시 은평구 서오릉로 58'
df.주소[20]='서울 강동구 양재대로96길 35'

In [302]:
df.to_csv('서울경찰서위치.csv',index=False)

문제 2

In [303]:
import requests
from urllib.parse import quote

In [304]:
with open('kakaoapikey.txt') as file:
    kakao_key = file.read()
len(kakao_key)

32

In [305]:
local_url='https://dapi.kakao.com/v2/local/search/address.json'
header = {'Authorization': f'KakaoAK {kakao_key}'}

In [306]:
lng_list, lat_list= [], []
for i in df.index:
    url = f'{local_url}?query={quote(df.주소[i])}'
    result=requests.get(url, headers=header).json()
    try:
        lng_list.append(float(result['documents'][0]['x']))
        lat_list.append(float(result['documents'][0]['y']))
    except:
        print(df.경찰서명[i])


In [307]:
df[df.경찰서명.isin(["서울서부경찰서","서울강동경찰서"])].주소

13    서울특별시 은평구 서오릉로 58
20    서울 강동구 양재대로96길 35
Name: 주소, dtype: object

In [308]:
url= f'{local_url}?query={quote("서울특별시 은평구 서오릉로 58")}'
result= requests.get(url,headers=header).json()
result['documents'][0]['x']

'126.923591244576'

In [309]:
url= f'{local_url}?query={quote("서울 강동구 양재대로96길 35")}'
result= requests.get(url,headers=header).json()
result['documents'][0]['x']

'127.139188584154'

In [310]:
lng_list,lat_list=[],[]
for i in df.index:
    url = f'{local_url}?query={quote(df.주소[i])}'
    result= requests.get(url, headers=header).json()
    try:
        lng_list.append(float(result['documents'][0]['x']))
        lat_list.append(float(result['documents'][0]['y']))
    except:
        print(df.경찰서명[i])

In [311]:
df['위도']=lat_list
df['경도']=lng_list
df.head(3)

,경찰서명,주소,위도,경도
0,경찰청 서울특별시경찰청 서울중부경찰서,서울특별시 중구 수표로 27,37.563632,126.990021
1,경찰청 서울특별시경찰청 서울종로경찰서,서울특별시 종로구 율곡로 46,37.575876,126.984551
2,경찰청 서울특별시경찰청 서울남대문경찰서,서울특별시 중구 한강대로 410,37.554811,126.973549


문제 3

In [148]:
import folium

In [313]:
map=folium.Map(location=[37.5502, 126.982], zoom_start=12)
for i in df.index:
    folium.Marker(
        location=[df.위도[i],df.경도[i]],
        tooltip=f'{df.경찰서명[i]}',
        popup=folium.Popup(df.주소[i], max_width=200),
        icon=folium.Icon(icon='glyphicon glyphicon-star')
    ).add_to(map)

title_html='<h3 align="center" style="font-size:20px">서울소재 경찰서 위치</h3>'
map.get_root().html
map